In [2]:
!pip install -U liblinear-official

In [17]:
from tqdm import tqdm
import numpy as np
from liblinear.liblinearutil import *
import pandas as pd
import ast
tqdm.pandas()
from sklearn.model_selection import train_test_split

In [10]:
CLASSES = [5, 7, 12, 19, 24, 26, 28, 36, 49, 83, 88, 119, 124, 136, 144, 157, 158, 169, 172]

In [18]:
class SVM():
    def __init__(self, cnn):
        # self.features = features # bounding box features
        self.cnn = cnn
        self.X = []
        self.Y = []
        self.num_classes = 894
        self.l = 1449

    def preprocess(self):
        all_labels = []
        for i in tqdm(range(self.l)):
            labelled = np.load("./nyudv2/label/" + str(i) + ".npy")
            unique_classes = np.unique(labelled)
            # print(unique_classes)
            temp = [0]*(self.num_classes + 1)
            for c in unique_classes:
                temp[c] = 1
            self.Y.append(temp)

    def process_X(self):
        df = pd.read_csv("RGBHHA_emb.csv")
        df["Embedding"] = df["Embedding"].progress_map(lambda x: ast.literal_eval(x))
        X_arr = df["Embedding"].values
        # final_X_arr = []
        # for i in tqdm(range(len(X_arr))):
        #     final_X_arr.append(ast.literal_eval(X_arr[i]))
        self.X_arr = X_arr
        return df
    
    def process_Y(self):
        Y = self.Y
        temp_Y = []
        for vec in Y:
            temp = []
            for idx in CLASSES:
                temp.append(vec[idx])
            temp_Y.append(temp)

        self.all_Y = temp_Y

    def train_test_split(self):
        X = self.X_arr
        y = self.all_Y

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    """
    def process_Y(self):
        temp_Y = {i: [0]*self.l for i in range(self.num_classes + 1)}

        for idx_image, i in enumerate(self.Y):
            for idx_class, j in enumerate(i):
                if j == 1:
                    # print(idx_class, idx_image)
                    temp_Y[idx_class][idx_image] = 1
        self.all_Y = temp_Y
    """

    def SVM_ind(self):
        X = self.X_train
        y = self.y_train
        # X = self.X_arr
        print(type(X), type(y))
        prob = problem(y, X)
        param = parameter('-B 10 -c 0.001 -b 1')
        m = train(prob, param)
        save_model("svm.model", m)
        self.m = m

        # train test split

In [19]:
svm = SVM("embeddings")

In [20]:
svm.preprocess()

100%|██████████| 1449/1449 [00:08<00:00, 178.35it/s]


In [21]:
svm.process_Y()

In [22]:
svm.all_Y

[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [23]:
df = svm.process_X()

  2%|▏         | 23/1449 [00:26<27:26,  1.15s/it]


KeyboardInterrupt: 

In [ ]:
svm.SVM_ind(svm.all_Y[1])

<class 'numpy.ndarray'> <class 'list'>
.....................*
optimization finished, #iter = 219
Objective value = -0.513852
nSV = 1181
